In [2]:
# Clone the NuPlan devkit repo
!git clone https://github.com/motional/nuplan-devkit.git
%cd nuplan-devkit

# Install the devkit in editable mode
!pip install -e .

Cloning into 'nuplan-devkit'...
remote: Enumerating objects: 5748, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 5748 (delta 326), reused 258 (delta 257), pack-reused 5267 (from 2)
Receiving objects: 100% (5748/5748), 3.96 MiB | 3.60 MiB/s, done.
Resolving deltas: 100% (3587/3587), done.
/Users/mariantemprana/Desktop/ISAC-code/nuplan-devkit
Obtaining file:///Users/mariantemprana/Desktop/ISAC-code/nuplan-devkit
  Preparing metadata (setup.py) ... done
  DEPRECATION: Legacy editable install of nuplan-devkit==1.2.2 from file:///Users/mariantemprana/Desktop/ISAC-code/nuplan-devkit (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more in

In [1]:
import os

# Set paths (adjust these for your local dataset)
os.environ["NUPLAN_DATA_ROOT"] = "../data/raw/NuPlan/train_pittsburgh"   # folder with *.db
print("Data root:", os.environ["NUPLAN_DATA_ROOT"])

Data root: ../data/raw/NuPlan/train_pittsburgh


In [ ]:
import sys, importlib, platform
mods = [
    "nuplan",
    "nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder",
    "nuplan.planning.scenario_builder.scenario_filter",
    "nuplan.common.actor_state.ego_state",
]
ok = True
for m in mods:
    try:
        importlib.import_module(m)
        print("OK  ", m)
    except Exception as e:
        ok = False
        print("FAIL", m, "->", repr(e))

print("Python:", sys.version.split()[0], "| Platform:", platform.platform())
assert ok, "One or more NuPlan modules failed to import."

OK   nuplan
OK   nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder
OK   nuplan.planning.scenario_builder.scenario_filter
OK   nuplan.common.actor_state.ego_state
Python: 3.10.18 | Platform: macOS-15.5-arm64-arm-64bit


In [42]:
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.metrics.utils.state_extractors import extract_ego_tire_steering_angle
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
import os

class DummyWorker:
    def __init__(self):
        self.number_of_threads = 0  # run everything in main thread

    def map(self, fn, inputs):
        return [fn(x) for x in inputs]

    def submit(self, fn, *args, **kwargs):
        return fn(*args, **kwargs)


# Adjust paths for your setup
data_root = os.environ["NUPLAN_DATA_ROOT"]
sensor_root = data_root   # usually same as data_root unless separated
db_files = ["../data/raw/NuPlan/train_pittsburgh/nuplan-v1.1/train/2021.08.17.13.10.50_veh-08_00122_00295.db"]   # replace with actual .db filename in your dataset folder
map_version = "nuplan-maps-v1.0"   # placeholder, not used if map_root=None

builder = NuPlanScenarioBuilder(
    data_root=data_root,
    sensor_root=sensor_root,
    db_files=db_files,
    map_version=map_version,
    map_root=None,   # No maps
)

# Build scenario filter with required args
scenario_filter = ScenarioFilter(
    scenario_types=None,
    scenario_tokens=None,
    log_names=None,
    map_names=None,
    num_scenarios_per_type=1,
    limit_total_scenarios=100,
    timestamp_threshold_s=15,
    ego_displacement_minimum_m=5,
    expand_scenarios=False,
    remove_invalid_goals=False,
    shuffle=False,
)

worker = DummyWorker()

# ---------------- Load scenarios ----------------
scenarios = builder.get_scenarios(scenario_filter, worker=worker)
print("Loaded", len(scenarios), "scenarios")

# ---------------- Inspect ego ----------------
sc = scenarios[0]

for i in range(sc.get_number_of_iterations()):
    ego = sc.get_ego_state_at_iteration(i)
    print(
        f"t={ego.time_point.time_s:.2f}s "
        f"x={ego.rear_axle.x:.2f} y={ego.rear_axle.y:.2f} "
        f"heading={ego.rear_axle.heading:.3f} rad "
        f"vx={ego.dynamic_car_state.rear_axle_velocity_2d.x:.2f} "
        f"vy={ego.dynamic_car_state.rear_axle_velocity_2d.y:.2f} "
        f"ax={ego.dynamic_car_state.rear_axle_acceleration_2d.x:.2f} "
        f"ay={ego.dynamic_car_state.rear_axle_acceleration_2d.y:.2f} "
        f"tire_angle={ego.tire_steering_angle:.3f} rad"
    )
    #break

Loaded 9 scenarios
t=1629206163.25s x=588576.06 y=4475797.49 heading=2.242 rad vx=11.53 vy=-0.15 ax=0.31 ay=-0.23 tire_angle=0.000 rad
t=1629206163.30s x=588575.70 y=4475797.94 heading=2.242 rad vx=11.54 vy=-0.16 ax=0.21 ay=-0.08 tire_angle=0.000 rad
t=1629206163.35s x=588575.36 y=4475798.37 heading=2.242 rad vx=11.54 vy=-0.16 ax=0.32 ay=0.10 tire_angle=0.000 rad
t=1629206163.40s x=588575.01 y=4475798.82 heading=2.242 rad vx=11.55 vy=-0.15 ax=0.30 ay=0.03 tire_angle=0.000 rad
t=1629206163.45s x=588574.66 y=4475799.26 heading=2.243 rad vx=11.56 vy=-0.16 ax=0.57 ay=0.06 tire_angle=0.000 rad
t=1629206163.50s x=588574.32 y=4475799.69 heading=2.243 rad vx=11.58 vy=-0.16 ax=0.64 ay=0.02 tire_angle=0.000 rad
t=1629206163.55s x=588573.97 y=4475800.13 heading=2.243 rad vx=11.60 vy=-0.15 ax=0.50 ay=0.01 tire_angle=0.000 rad
t=1629206163.60s x=588573.62 y=4475800.57 heading=2.243 rad vx=11.62 vy=-0.15 ax=0.48 ay=0.00 tire_angle=0.000 rad
t=1629206163.65s x=588573.27 y=4475801.02 heading=2.243 rad

In [20]:
pip install pyquaternion matplotlib pytest scipy opencv-python requests tqdm numpy pandas shapely pyproj geopandas aioboto3 aiobotocore boto3 retry rasterio omegaconf hydra-core

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install shapely pyproj geopandas

  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 5.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 4.8 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 5.0 MB/s  0:00:03m0:00:0100:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [geopandas]/8 [pandas]]
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install "torch==2.3.*" torchvision --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 5.0 MB/s  0:00:12m0:00:0100:01
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.3 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.8 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.3 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s  0:00:00 eta 0:00:01
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.5 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 4.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━